In [225]:
import numpy as np
import torch
import torch.nn as nn
import torch.optim as optim
from sklearn.cluster import SpectralClustering
import scipy.sparse as sp
from torch.nn.functional import normalize
import torch.nn.functional as F
from sklearn import metrics as metrics_sk

In [279]:
def sigmoid(array):
    for i in range(len(array)):
        for j in range(len(array[i])):
            array[i][j] = 1/(1 + np.exp(-array[i][j]))
    return array

In [143]:
def matrix_partition(data,adjacency_matrix,n_partitions):
    sc = SpectralClustering(n_clusters=n_partitions, affinity='precomputed', random_state=42)
    labels = sc.fit_predict(adjacency_matrix)
    sub_adjacency_matrices = []
    data_matrices = []
    for label in np.unique(labels):
        sub_adjacency_matrix = adjacency_matrix[labels == label][:, labels == label]
        sub_adjacency_matrices.append(sub_adjacency_matrix)
        data_matrix = data[:,:,labels == label,:]
        data_matrices.append(data_matrix.double())
    #sub_adjacency_matrices.append(adjacency_matrix)
    #data_matrices.append(data)    
    
    
    return sub_adjacency_matrices, data_matrices, labels

In [144]:
def data_reconstruct(labels,data_matrices,seq_len,num_nodes):

    unique_labels = np.unique(labels)
    k = len(unique_labels)

    n = seq_len
    m = num_nodes

    recovered_matrix = np.zeros((n, m))
    for i, small_matrix in enumerate(data_matrices):
        indices = np.where(labels == unique_labels[i])[0]
        recovered_matrix[:,indices] = small_matrix
        
    return recovered_matrix


In [145]:
def calculate_laplacian_with_self_loop(matrix):
    matrix = matrix + torch.eye(matrix.size(0))
    row_sum = matrix.sum(1)
    d_inv_sqrt = torch.pow(row_sum, -0.5).flatten()
    d_inv_sqrt[torch.isinf(d_inv_sqrt)] = 0.0
    d_mat_inv_sqrt = torch.diag(d_inv_sqrt)
    normalized_laplacian = (
        matrix.matmul(d_mat_inv_sqrt).transpose(0, 1).matmul(d_mat_inv_sqrt)
    )
    return normalized_laplacian

In [200]:
class GCNLayer(nn.Module):
    def __init__(self, in_features, out_features):
        super(GCNLayer, self).__init__()
        self.linear = nn.Linear(in_features, out_features)
    
    def forward(self, input, adj_matrix):
        x = input.double()
        adj = adj_matrix.double()
        x = torch.matmul(adj, x)
        x = self.linear(x)
        x = F.relu(x)
        return x

In [210]:
X_data = np.load("/Users/pipipu/Desktop/HAGEN/HAGEN-code/crime-data/CRIME-LA/8/train.npz")
X = torch.from_numpy(X_data['x'])
X = X.permute(0,3,2,1)

y = torch.from_numpy(X_data['y'])
y = y.permute(0,3,2,1)

adj = torch.from_numpy(np.load('/Users/pipipu/Desktop/HAGEN/HAGEN-code/crime-data/sensor_graph/adj_mx_la.pkl',allow_pickle = True)[2])

In [202]:
n_partitions = 2
sub_adjacency_matrices, data_matrices, labels = matrix_partition(X,adj,n_partitions)

In [351]:
class Spatial_Block(torch.nn.Module):
    def __init__(self, in_features, out_features, num_graphs):
        super(Spatial_Block, self).__init__()  
        self.num_graphs = num_graphs
        self.in_features = in_features
        self.out_features = out_features
        self.GCN_Layers = nn.ModuleList([
            GCNLayer(self.in_features, self.out_features) for _ in range(num_graphs)
        ])
        self.whole = GCNLayer(self.in_features, self.out_features)

        
        
    def forward(self, data_matrices, sub_adjacency_matrices, x, adj):
        
        res = x
        output = self.whole(x,adj)
        
        processed_output = []
        for i in range(self.num_graphs):
            spatial_layer = self.GCN_Layers[i]
            x = data_matrices[i]
            adj = sub_adjacency_matrices[i]
            x = x.double()
            adj = adj.double()
            x = spatial_layer(x, adj)
            processed_output.append(x)
        
        processed_output = torch.cat(processed_output, dim=-2)
        
        x = output + processed_output + res

        #print(x.shape)
        return output


In [352]:
class Temporal_Block(torch.nn.Module):
    def __init__(self, in_features, out_features):
        super(Temporal_Block, self).__init__()
        self.linear = nn.Linear(in_features, out_features)

    def forward(self, x):
        x = self.linear(x)
        return x


In [353]:
class Frame(torch.nn.Module):
    def __init__(self,in_feature_spatial, out_feature_spatial, num_graph, in_feature_temporal, out_feature_temporal):
        super(Frame, self).__init__()
        self.spatial = Spatial_Block(in_feature_spatial,out_feature_spatial,num_graph)
        self.temporal = Temporal_Block(in_feature_temporal,out_feature_temporal)
        

    def forward(self,data_matrices,sub_adjacency_matrices,x, adj):
        x = self.spatial(data_matrices, sub_adjacency_matrices,x, adj)
        x = self.temporal(x)
        return x

In [354]:
def cross_entropy(preds, labels):
    lossfn = nn.BCELoss()
    predss = torch.sigmoid(preds)
    loss = lossfn(predss, labels)
    return loss

In [356]:
model = Frame(8,8,2,8,1).double()
learning_rate = 1e-2
optimizer = optim.SGD(model.parameters(), lr=learning_rate)
model = model
num_epochs = 100
quantile = 0.1
X = X.double()
y = y.double()
criterion = nn.L1Loss()

for epoch in range(num_epochs):
    running_loss = 0
    running_macro_f1 = 0
    running_micro_f1 = 0
    for i in range(X.shape[0]):
        small_data_matrices = []
        for j in range(n_partitions):
             small_data_matrices.append(data_matrices[j][i,:,:,:])
        
       
        output = model(small_data_matrices,sub_adjacency_matrices,X[i,:,:,:],adj)
        labels = y[i,:,:,:]
        
        #output = output.reshape(1,-1)
        #labels = labels.reshape(1,-1)
        loss = cross_entropy(output, labels)
       
            # Backward pass and optimization
        optimizer.zero_grad()
        loss.backward()
        optimizer.step()
        
            # Update the running loss
        running_loss += loss.item()
        output = sigmoid(output.detach().numpy())
        
        output[output>0.5]=1
        output[output<=0.5]=0
        output = output.reshape(113,8)
        labels = labels.reshape(113,8)
        macro_f1 = metrics_sk.f1_score(labels.detach().numpy(),output, average = 'macro',zero_division=1)
        micro_f1 = metrics_sk.f1_score(labels.detach().numpy(), output, average = 'micro',zero_division=1)
        running_macro_f1 += macro_f1
        running_micro_f1 += micro_f1
        # Print the average loss for the epoch
    epoch_loss = running_loss / X.shape[0]
    running_macro_f1 = running_macro_f1 / X.shape[0]
    running_micro_f1 = running_micro_f1 / X.shape[0]

    print(f"Epoch {epoch+1}/{num_epochs}, Loss: {epoch_loss}, macro_f1: {running_macro_f1}, micro_f1: {running_micro_f1}")


    
       

Epoch 1/100, Loss: 0.6614832347552604, macro_f1: 0.004206751042168665, micro_f1: 0.004220258770828376
Epoch 2/100, Loss: 0.6335882582657298, macro_f1: 0.04668143819724827, micro_f1: 0.04696927195960932
Epoch 3/100, Loss: 0.625309767970725, macro_f1: 0.09923587076618191, micro_f1: 0.09981613697542416
Epoch 4/100, Loss: 0.6188079291576312, macro_f1: 0.14550009601631467, micro_f1: 0.1461908447184852
Epoch 5/100, Loss: 0.6135812160012716, macro_f1: 0.1774175376878689, micro_f1: 0.17820157972103173
Epoch 6/100, Loss: 0.6093184611306529, macro_f1: 0.19979194370646858, micro_f1: 0.2005190712277339
Epoch 7/100, Loss: 0.6057990543912174, macro_f1: 0.21988795112941611, micro_f1: 0.22040077260747856
Epoch 8/100, Loss: 0.602851014577147, macro_f1: 0.2313671391730587, micro_f1: 0.23188588765821638
Epoch 9/100, Loss: 0.6003849144836482, macro_f1: 0.2404060323819355, micro_f1: 0.2409863519838392
Epoch 10/100, Loss: 0.598280188333861, macro_f1: 0.24816169261223306, micro_f1: 0.24870466440859162
Epoch 

In [228]:
test_data = np.load('/Users/pipipu/Desktop/HAGEN/HAGEN-code/crime-data/CRIME-LA/8/train.npz')
X_test = torch.from_numpy(test_data['x'])
y_test = torch.from_numpy(test_data['y'])
X_test = X_test.permute(0,3,2,1)
y_test = y_test.permute(0,3,2,1)
